Let's export the trained model to HuggingFace Hub in safetensors formats for compatibility with downstream inference engines. First, we'll define some variables.

In [ ]:
model_name = "UltraZoom-2X"
checkpoint_path = "./checkpoints/checkpoint.pt"
exports_path = "./exports"

Then, we'll load the base model checkpoint into memory from disk.

In [ ]:
import torch

from src.ultrazoom.model import UltraZoom

checkpoint = torch.load(checkpoint_path, map_location="cpu", weights_only=True)

model = UltraZoom(**checkpoint["model_args"])

model.add_weight_norms()

model = torch.compile(model)

model.load_state_dict(checkpoint["model"])

model.remove_parameterizations()

model.eval()

print("Base checkpoint loaded successfully")

Now, let's export the model in HuggingFace format so that it can be used with the HuggingFace ecosystem.

In [ ]:
from os import path

hf_path = path.join(exports_path, model_name)

model.save_pretrained(hf_path)

print(f"Model saved to {hf_path}")

Next, we'll login to HuggingFaceHub and upload the model under our account.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

model.push_to_hub(model_name)

Lastly, we'll export a model in ONNX format for use with the ONNX runtime.

In [ ]:
from os import path

from torch.onnx import export as export_onnx

onnx_path = path.join(exports_path, model_name, "model.onnx")

example_inputs = torch.randn(1, 3, 128, 128)

onnx_model = export_onnx(model, example_inputs, dynamo=True)

onnx_model.save(onnx_path)